# GC Subscription Predict Model


**Project Objective:**
criação de um book de variáveis (feature store) e posteriormente criar a nossa variável resposta (target), i.e. aquilo que queremos prever.

Qual a probabilidade de uma pessoa assinar a GC nos próximos 15 dias?

## Project Solution Planning

### Input

1. Problema de Negócio
    - ...
    - ...
2. Conjunto de dados

    

### Output


1. ...
             
2. Relatório com as respostas das perguntas de negócio

    - ...
    - ...
    - ...
    - ...
    - ...
    - ...
    - ...
    - ...
    - ...

### Tasks

1. 
2. 
3. 
4. 

# Imports

In [1]:
import pandas as pd
import numpy  as np
import sqlalchemy

import matplotlib.pyplot as plt

from sklearn import metrics, model_selection
from sklearn import ensemble
from sklearn import tree
from sklearn import linear_model
from sklearn import pipeline

from feature_engine import imputation
from feature_engine import encoding
from random         import random

import scikitplot as skplt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,\
confusion_matrix, matthews_corrcoef, make_scorer, roc_curve, precision_recall_curve

## Helper Functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
# Model's performance function on training dataset
def performance(model, x_train, y_train):
    # define scoring metrics
    scoring = {'accuracy': 'accuracy',
               'precision': make_scorer(precision_score),
               'recall': make_scorer(recall_score),
               'f1_score': make_scorer(f1_score),
               'roc_auc_score': make_scorer(roc_auc_score),
               'mcc': make_scorer(matthews_corrcoef)}

    # calculate scores with cross_validate
    scores = cross_validate(model, x_train, y_train, cv=10, scoring=scoring)
    
    # performance data frame
    performance = pd.DataFrame.from_dict(scores).drop(['fit_time', 'score_time'], axis=1)
    performance = pd.DataFrame(performance.mean()).T
    return performance

In [3]:
jupyter_settings()
pd.set_option('display.max_columns', None)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/marxcerqueira/.pyenv/versions/3.9.2/envs/gc-prediction/lib/python3.9/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


NameError: name 'HTML' is not defined

# Loading and Splitting Data

In [4]:
# conect with GC.db and get ABT
conn = sqlalchemy.create_engine("sqlite:///../../../data/gc.db")
df = pd.read_sql_table('tb_abt_sub', conn)

In [30]:
# Spliting database Train-Validation-Test(81,9,10) and Out of Time

df_oot = df[df['dtReff'].isin(['2022-01-15', '2022-01-16'])].copy()
df_train = df[~df['dtReff'].isin(['2022-01-15', '2022-01-16'])].copy()

#features and target list
features = df_train.columns.tolist()[2:-1]
target = 'flagSub'


#define df_train, df_test (80/20)
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_train[features],
                                                                    df_train[target],
                                                                    random_state=42,
                                                                    test_size=0.1)

# train valid split
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X_train, y_train, test_size=0.10, random_state=42)

# x y out of time base
X_oot = df_oot[features].copy()
y_oot = df_oot[target].copy()


# check balance of classes (0, 1) among y_train and y_test.
print(f"Class size for train dataset: {np.bincount(y_train)}")
print(f"Class size for valid dataset: {np.bincount(y_valid)}")
print(f"Class size for test dataset: {np.bincount(y_test)}")
print(f"Class size for OOT dataset: {np.bincount(y_oot)}")
print(f'% of data in train/valid/test: {(X_train.shape[0]/df_train.shape[0])*100}/{(X_valid.shape[0]/df_train.shape[0])*100}/{(X_test.shape[0]/df_train.shape[0])*100}')



Class size for train dataset: [77097  5654]
Class size for valid dataset: [8591  604]
Class size for test dataset: [9504  713]
Class size for OOT dataset: [2427   91]
% of data in train/valid/test: 80.99899180721005/9.000323013223966/10.000685179565988


In [58]:
# 0 and 1 proportions
print(y_train.value_counts(normalize = True)*100)
print(y_valid.value_counts(normalize = True)*100)
print(y_test.value_counts(normalize = True)*100)
print(y_oot.value_counts(normalize = True)*100)

0    93.167454
1     6.832546
Name: flagSub, dtype: float64
0    93.431213
1     6.568787
Name: flagSub, dtype: float64
0    93.021435
1     6.978565
Name: flagSub, dtype: float64
0    96.386021
1     3.613979
Name: flagSub, dtype: float64


# DATA DISCRIPTION

## Data Dimension

In [39]:
print(f'\nTrain base:\nNumber of rows:{X_train.shape[0]} ')
print(f'Number of columns:{X_train.shape[1]} ')

print(f'\nValidation base:\nNumber of rows:{X_valid.shape[0]} ')
print(f'Number of columns:{X_valid.shape[1]} ')

print(f'\nTest base:\nNumber of rows:{X_test.shape[0]} ')
print(f'Number of columns:{X_test.shape[1]} ')


Train base:
Number of rows:82751 
Number of columns:75 

Validation base:
Number of rows:9195 
Number of columns:75 

Test base:
Number of rows:10217 
Number of columns:75 


## Data Types

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104681 entries, 0 to 104680
Data columns (total 78 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   dtReff                 104681 non-null  object 
 1   idPlayer               104681 non-null  int64  
 2   qtPartidas             104681 non-null  int64  
 3   qtPartidasMenor16      104681 non-null  int64  
 4   qtDias                 104681 non-null  int64  
 5   qtDiasUltimaLobby      104681 non-null  float64
 6   mediaPartidaDias       104681 non-null  float64
 7   avgQtKill              104681 non-null  float64
 8   avgQtAssist            104681 non-null  float64
 9   avgQtDeath             104681 non-null  float64
 10  avgKDA                 104681 non-null  float64
 11  KDAgeral               104681 non-null  float64
 12  avgKARound             104681 non-null  float64
 13  KARoundgeral           104681 non-null  float64
 14  avgQtHs                104681 non-nu

## Check NA Values

In [42]:
df.isna().sum()

dtReff                   0
idPlayer                 0
qtPartidas               0
qtPartidasMenor16        0
qtDias                   0
                     ...  
flTwitch                 0
descCountry              0
vlIdadePlayer        21625
vlDiasCadastro           0
flagSub                  0
Length: 78, dtype: int64

In [44]:
# separate numerical and categorical features into lists
cat_features = X_train.dtypes[X_train.dtypes=='object'].index.tolist()
num_features = list(set(X_train.columns) - set(cat_features))

In [46]:
print("Estatística de missings")
print("Missing numerico:")
is_na = X_train[num_features].isna().sum()
print(is_na[is_na>0])


Estatística de missings
Missing numerico:
WinRateNuke        41942
WinRateMirage      13717
WinRateDust2       29072
WinRateAncient     62094
WinRateTrain       50145
vlIdadePlayer      16993
WinRateVertigo     36173
WinRateOverpass    37692
WinRateInferno     17432
dtype: int64


In [47]:
print("Estatística de missings")
print("Missing cat:")
is_na = X_train[cat_features].isna().sum()
print(is_na[is_na>0])

Estatística de missings
Missing cat:
Series([], dtype: int64)


## Fillout NA

In [48]:
missing_1 = ["vlIdade",
             "winRateDust2",
             "winRateNuke",
             "winRateOverpass",
             "winRateVertigo",
             "winRateTrain",
             "winRateMirage",
             "winRateInferno",
             "winRateAncient"]

## Descriptive Statistics

In [49]:
X_train.describe()

,qtPartidas,qtPartidasMenor16,qtDias,qtDiasUltimaLobby,mediaPartidaDias,avgQtKill,avgQtAssist,avgQtDeath,avgKDA,KDAgeral,...,vlLevelAtual,qtMedalDistict,qtMedalhaAdquiridas,qtPremium,qtPlus,flFacebook,flTwitter,flTwitch,vlIdadePlayer,vlDiasCadastro
count,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,...,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,82751.000000,65758.000000,82751.000000
mean,20.763459,0.458109,8.060084,7.150226,2.150702,18.117899,3.676167,18.714680,1.244268,1.182613,...,11.401554,0.120808,0.002066,0.044640,0.103830,0.021559,0.017825,0.020399,23.653463,1056.969529
std,24.978140,1.178337,6.887136,8.033700,1.051248,4.878872,1.183652,2.833243,0.389574,0.339953,...,4.896939,0.334073,0.045411,0.246378,0.373534,0.145238,0.132314,0.141360,5.390378,661.303348
min,1.000000,0.000000,1.000000,0.000012,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.040000,0.040197
25%,4.000000,0.000000,2.000000,0.939057,1.400000,15.500000,3.041667,17.478261,1.035808,1.003192,...,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.900000,474.677517
50%,11.000000,0.000000,6.000000,3.803507,2.000000,18.200000,3.666667,18.851064,1.225335,1.170507,...,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.430000,972.193322
75%,28.000000,1.000000,12.000000,11.199890,2.625000,20.923077,4.222222,20.000000,1.418487,1.340795,...,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.717500,1616.181869
max,262.000000,37.000000,30.000000,29.999375,12.000000,45.000000,14.000000,44.000000,9.250000,9.250000,...,21.000000,2.000000,1.000000,4.000000,5.000000,1.000000,1.000000,1.000000,60.460000,3133.583958


## Multivariate Analysis

# DATA PREPARATION

## Feature Transform

In [50]:
#imputação dos dados
input_1 = imputation.ArbitraryNumberImputer(arbitrary_number = -1, variables =missing_1 )

# one hot enconding
onehot = encoding.OneHotEncoder(drop_last = True, variables=cat_features)


# FEATURE SELECTION

## Feature Selector (boruta?)

In [60]:
# train random forest classifier
rf_clf = ensemble.RandomForestClassifier(n_estimators = 300, min_samples_leaf=50, n_jobs =-1, random_state = 42)
rf_clf.fit(X_train, y_train)

# feature importance data frame
feat_imp = pd.DataFrame({'feature': X_train.columns,
                        'feature_importance': rf.feature_importances_})\
                        .sort_values('feature_importance', ascending=False)\
                        .reset_index(drop=True)


# plot feature importance
plt.subplots(figsize=(12,6))
sns.barplot(x='feature_importance', y='feature', data=feat_imp, orient='h', color='royalblue')\
    .set_title('Feature Importance');

/home/marxcerqueira/.pyenv/versions/3.9.2/envs/gc-prediction/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.
  warnings.warn(


ValueError: could not convert string to float: 'ar'

## Best Features

## Spliting dataframe into training and test dataset

0    93.167454
1     6.832546
Name: flagSub, dtype: float64
0    93.431213
1     6.568787
Name: flagSub, dtype: float64
0    93.021435
1     6.978565
Name: flagSub, dtype: float64
0    96.386021
1     3.613979
Name: flagSub, dtype: float64


In [55]:
y_valid.value_counts(normalize = True)*100

0    93.431213
1     6.568787
Name: flagSub, dtype: float64

In [52]:
y_test.value_counts(normalize = True)*100

0    93.021435
1     6.978565
Name: flagSub, dtype: float64

In [56]:
y_oot.value_counts(normalize = True)*100

0    96.386021
1     3.613979
Name: flagSub, dtype: float64

# MACHINE LEARNING ALGORITHM MODELS

In [51]:
## MODEL
rf_clf = ensemble.RandomForestClassifier(n_estimators=200,
                                         min_samples_leaf=50,
                                         n_jobs = -1,
                                         random_state=42)

ada_clf = ensemble.AdaBoostClassifier(n_estimators=200,
                                      learning_rate=0.8,
                                      random_state=42)

et_clf = ensemble.ExtraTreesClassifier(n_estimators=200,
                                      max_depth=15,
                                      min_samples_leaf=50,
                                      n_jobs=-1,
                                      random_state=42)

dt_clf = tree.DecisionTreeClassifier(max_depth=15,
                                     min_samples_leaf=50,
                                     random_state=42)

lr_clf = linear_model.LogisticRegressionCV(cv=4, n_jobs=-1)

## Modelo 1

## Modelo 2

## Modelo 3

## Compare Model's Performance

# HYPERPARAMETERS FINE TUNING

# ERROR INTERPRETATION

# MODEL DEPLOYMENT